# 1. Imports

In [4]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt

import torch
import utils
display = utils.notebook_init()
import torch
import numpy as np
import cv2
import keyboard
import win32gui
from PIL import ImageGrab
import pandas as pd
import os

print("Is CUDA available: %s" % torch.cuda.is_available()) 
print("GPUs: %s" % torch.cuda.device_count()) 
print("GPU name: %s" % torch.cuda.get_device_name(0))

YOLOv5  v7.0-117-g85f6019 Python-3.9.13 torch-1.8.2+cu111 CUDA:0 (NVIDIA GeForce GTX 1660 Ti, 6144MiB)


Setup complete  (6 CPUs, 15.9 GB RAM, 209.0/222.4 GB disk)
Is CUDA available: True
GPUs: 1
GPU name: NVIDIA GeForce GTX 1660 Ti


# 2. Load model

In [7]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='../weights/final_weights.pt', force_reload=True)

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to C:\Users\julen/.cache\torch\hub\master.zip
YOLOv5  v7.0-117-g85f6019 Python-3.9.13 torch-1.8.2+cu111 CUDA:0 (NVIDIA GeForce GTX 1660 Ti, 6144MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


# 3. Real time detections and executing actions

In [4]:
"""
Function to convert the output results into two np.arrays, one for cuphead and one for goopy
"""
def convert_results(results):
    cup_det = np.zeros(5)
    goop_det = np.zeros(5)

    res = results.pandas().xyxy[0]

    class_num = list(res["class"].astype(int))

    if(len(class_num) == 2):
        xmax = res.xmax.astype(float)
        xmin = res.xmin.astype(float)
        ymax = res.ymax.astype(float)
        ymin = res.ymin.astype(float)

        x = list(((xmax+xmin)/2)/1168)
        y = list(((ymax+ymin)/2)/657)
        w = list((xmax-xmin)/1168)
        h = list((ymax-ymin)/657)
        
        if(class_num[0]==0):
            cup_det = np.array([class_num[0], x[0], y[0], w[0], h[0]])
            goop_det = np.array([class_num[1], x[1], y[1], w[1], h[1]])
        else:
            goop_det = np.array([class_num[0], x[0], y[0], w[0], h[0]])
            cup_det = np.array([class_num[1], x[1], y[1], w[1], h[1]])
    
    return cup_det, goop_det

In [5]:
DIST = 0.15
AREA_THR = 0.05
LEFT_LIMIT = 0.15
RIGHT_LIMIT = 0.85

cup = np.empty(5)
goopy = np.empty(5)

key = 'space'

keyboard.wait('s') # wait for the 's' key to be pressed to start
print("Started")

while True:
    keyboard.press('x') # keep the shooting key pressed

    hwnd = win32gui.FindWindow(None, "Cuphead") # find the game's window
    rect = win32gui.GetWindowPlacement(hwnd)[-1]
    image = ImageGrab.grab(rect)
    image = image.crop((6, 30, 1174, 687)) # crop the frame for deletting the top bar and some pixels on the sides

    # Make detections 
    results = model(image)

    # convert results to the yolo input format 
    cup, goopy = convert_results(results)

    cup_x = cup[1]
    goop_x = goopy[1]

    goopy_w = goopy[3]
    goopy_h = goopy[4]
    
    if(goopy[1] != 0): # check if there are only 2 detections
        # if cuphead is beyond the left or right limits then dash to the opposite side
        if(cup_x <= LEFT_LIMIT):
            keyboard.release('left')
            key = 'right'
            keyboard.press_and_release('right+left_shift')
        elif(cup_x >= RIGHT_LIMIT):
            keyboard.release('right')
            key = 'left'
            keyboard.press_and_release('left+left_shift')

        # face to the enemy for shooting them
        if(cup_x < goop_x):
            if(key != 'right'):
                keyboard.release('left')
                key = 'right'
                keyboard.press_and_release('right')
        elif(cup_x > goop_x):
            if(key != 'left'):
                keyboard.release('right')
                key = 'left'
                keyboard.press_and_release('left')  

        # if goopy's area is bigger than the area threshold go down
        if(goopy_w*goopy_h >= AREA_THR or goop_x < LEFT_LIMIT or goop_x > RIGHT_LIMIT):
            keyboard.press('down')
        else:
            keyboard.release('down')

        # if cuphead and goopy are close then dash to avoid damage
        if(abs(cup_x-goop_x) < DIST): 
            if(cup_x < goop_x):
                keyboard.release('left')
                keyboard.press_and_release('right+left_shift') # press the dashing key sequence
            elif(cup_x > goop_x):
                keyboard.release('right')
                keyboard.press_and_release('left+left_shift') # press the dashing key sequence           
    
    cv2.imshow('YOLO', cv2.cvtColor(np.squeeze(results.render()), cv2.COLOR_BGR2RGB)) # show the frame with the detections in rgb format 

    # when exiting make sure that all the keys are released
    if cv2.waitKey(10) & 0xFF == ord('q'):
        print("Q pressed, exiting")
        keyboard.release('x')
        keyboard.release('shift')
        keyboard.release('right')
        keyboard.release('left')
        keyboard.release('down')
        break
            
cv2.destroyAllWindows()

Started
Q pressed, exiting
